# An example file on how to use R within jupyter notebooks
- we first look at importing functions, then how to operate with bigquery

In [ ]:
getwd()
#this will just show you what directory you are in

In [ ]:
# These are the packages you need to install to get pre commits to work... should only need to be done once...
install.packages('docopt')
install.packages('git2r')
install.packages('styler')

In [ ]:
#if you are importing functions into a file outside of fintrans_toolbox you will have to import like this
source('~/fintrans_toolbox/src/example_R_functions.r')


In [ ]:
#example of how to use the function from example_R_functions.R
print_x('boo')

In [ ]:
#example loading in a package
library(data.table) 

In [ ]:
import pandas as pd  # this doesnt work cos its python.

In [ ]:
# hello hello hello hello hello hello more than 88 characters what happpens hellloooooooooooooooooooooooooooooo

In [ ]:
library(bigrquery)
library(bigQueryR)
library(dplyr)

https://www.infoworld.com/article/3622926/how-to-use-r-with-bigquery.html

This document is the most helpful with how to use R with big query. 

In [ ]:
library(bigrquery)

In [ ]:
con <- dbConnect(
    bigquery(),
    project = 'ons-fintrans-data-prod',
    dataset = 'fintrans_visa'
    )

In [ ]:
dbListTables(con)

In [ ]:
con1 <- dbConnect(
    bigquery(),
    project = 'ons-fintrans-analysis-prod',
    dataset = 'fin_wip_notebook'
    )

In [ ]:
dbListTables(con1)

In [ ]:
sml <- tbl(con1, "sml_cardholders_q")

In [ ]:
str(sml)

In [ ]:
glimpse(sml)

In [ ]:
df <- select(sml, date_time) %>% distinct() %>% collect()

# need to add collect to view things

In [ ]:
df

In [ ]:
select(sml, date_time) %>% distinct() %>% show_query()

In [ ]:
con <- dbConnect(
    bigquery(),
    project = 'ons-fintrans-analysis-prod',
    dataset = 'fintrans_visa'
    )

# Note, we keep the project as 'ons-fintrans-analysis-prod' not 'ons-fintrans-data-prod' even when doing SQLs like the one below
# where we retrieve data from 'ons-fintrans-data-prod'

In [ ]:
# or running SQL From within R
sql <- "SELECT * FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location LIMIT 10"
library(DBI)
my_results <- dbGetQuery(con, sql)

In [ ]:
my_results